- why for alpha=1.0 (in the code) the performance of ens and prod is not more or less the same for M=1 (i.e. ensembles of size 1)? the performance gap is actually the biggest for alpha=1, why??
- study pretraining in more details: have a plot where you first pretrain DE for 50 epochs and then switch to PoE objective (do it once with alpha=0, i.e. so only have product in the objective function, and once with alpha=0.5) for various types of epochs (10, 20, 30, 40, 50)
- in the members_nll part of PoE objective function we should probably reuse individual likelihoods from the product_nll term, or? in the current implementation, we have "standard" categorical distribution in members_nll while in product_nll we have hard_ovr distribution...
- stochastic sampling of ensemble members during training

DONE:
- plot effect of alpha and pretraining
- random seed in ensembles -> are we using different seeds for each member? YES
- study magnitudes of terms in objective function (would imagine for that sum of expert likelihoods is larger compared to PoE likelihood), how does this impact the choice of alpha?

In [1]:
import os
import logging
from functools import partial

from jax import random
from torchvision import transforms
import wandb
from flax.training.checkpoints import save_checkpoint, restore_checkpoint

from src.models import make_Hard_OvR_Ens_loss as make_prod_loss
from src.models import make_Hard_OvR_Ens_MNIST_plots as make_plots
from src.models import make_Cls_Ens_loss as make_ens_loss
from src.data import get_image_dataset, NumpyLoader
from src.utils.training import setup_training, train_loop
from src.utils.notebook_metrics import *
from experiments.configs.mnist_hard_ovr_classification import get_config

from operator import itemgetter
from  functools import partial
from itertools import combinations

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from plotly.subplots import make_subplots
import plotly.graph_objects as go

2022-10-14 16:30:22.095122: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-14 16:30:22.769081: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-14 16:30:22.769160: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-10-14 16:30:22.769168: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [2]:
os.environ['XLA_FLAGS'] = "--xla_gpu_force_compilation_parallelism=1"

In [3]:
os.environ['WANDB_NOTEBOOK_NAME'] = 'mnist_performance_gap_investigation.ipynb'
# ^ W&B doesn't know how to handle VS Code notebooks.

wandb.login()

wandb: Currently logged in as: metodj. Use `wandb login --relogin` to force relogin


True

In [4]:
config = get_config()

In [5]:
ens_config = config.copy_and_resolve_references()
ens_config.model_name = 'Cls_Ens'
del ens_config.β_schedule

### 1) load datasets

In [6]:
train_dataset, _, val_dataset = get_image_dataset(
    dataset_name=config.dataset_name,
    val_percent=config.val_percent,
    flatten_img=True,
    train_augmentations=[
        # transforms.RandomCrop(28, padding=2),
        # transforms.RandomAffine(degrees=5, translate=(0.05, 0.05), scale=(0.95, 1.05), shear=5),
        # transforms.RandomHorizontalFlip(),
        # transforms.
    ]
)
ens_train_loader = NumpyLoader(train_dataset, config.batch_size, num_workers=8)
ens_val_loader = NumpyLoader(val_dataset, config.batch_size, num_workers=8)

train_dataset, test_dataset, val_dataset = get_image_dataset(
    dataset_name=config.dataset_name,
    val_percent=config.val_percent,
    flatten_img=True,
    train_augmentations=[
        # transforms.RandomCrop(28, padding=2),
        # transforms.RandomAffine(degrees=5, translate=(0.05, 0.05), scale=(0.95, 1.05), shear=5),
        # transforms.RandomHorizontalFlip(),
        # transforms.
    ]
)
train_loader = NumpyLoader(train_dataset, config.batch_size, num_workers=8)
val_loader = NumpyLoader(val_dataset, config.batch_size, num_workers=8)
test_loader = NumpyLoader(test_dataset, config.batch_size, num_workers=8)

In [7]:
print(ens_config)

batch_size: 512
dataset_name: MNIST
epochs: 50
learning_rate: 0.003
lr_schedule:
  decay_steps: 5300
  end_value: 0.00030000000000000003
  peak_value: 0.009000000000000001
  warmup_steps: 1060
lr_schedule_name: warmup_cosine_decay_schedule
model:
  learn_weights: false
  net:
    depth: 5
    hidden_size: 200
    out_size: 10
    p_drop: 0.1
  size: 5
model_name: Cls_Ens
optim:
  momentum: 0.9
  weight_decay: 0.0001
optim_name: sgdw
val_percent: 0.1



In [14]:
print(config)

alpha: 1.0
batch_size: 512
dataset_name: MNIST
epochs: 50
learning_rate: 0.003
lr_schedule:
  decay_steps: 5300
  end_value: 0.00030000000000000003
  peak_value: 0.009000000000000001
  warmup_steps: 1060
lr_schedule_name: warmup_cosine_decay_schedule
model:
  learn_weights: false
  net:
    depth: 5
    hidden_size: 200
    out_size: 10
    p_drop: 0.1
  size: 5
model_name: Hard_OvR_Ens
optim:
  momentum: 0.9
  weight_decay: 0.0001
optim_name: sgdw
pretrained: false
val_percent: 0.1
"\u03B2_schedule":
  end: 16.0
  name: sigmoid
  start: 2.0
  steps: 5300



### 2) model training

In [11]:
RANDOM_SEEDS = [0]
# ALPHAS = [0., 0.2, 0.4, 0.5, 0.6, 0.8 ,1.]
# ALPHAS = [0., 1/6, 0.5, 5/6, 1.]
ALPHAS = [1.]
RECOMPUTE_ENS = False

In [13]:
for i in RANDOM_SEEDS:
    rng = random.PRNGKey(i)
    setup_rng, rng = random.split(rng)
    init_x = train_dataset[0][0]
    init_y = train_dataset[0][1]

    ens_model, state = setup_training(ens_config, setup_rng, init_x, init_y)
    print(rng)
    print(state.params)

    if RECOMPUTE_ENS:
        _, ens_state = train_loop(
            ens_model, state, ens_config, rng, make_ens_loss, make_ens_loss, ens_train_loader, ens_val_loader,
            # test_loader,
            wandb_kwargs={
                'mode': 'online',
                # 'notes': 'Data augmentation',
                # 'tags': ['MNIST testing'],
            },
            # plot_fn=make_plots,
            # plot_freq=1,
        )

        save_checkpoint(f'dynNN_redux/ens_model_{i}', ens_state, 1, overwrite=True)

    rng = random.PRNGKey(i)
    setup_rng, rng = random.split(rng)
    
    for alpha in ALPHAS:
        # for pretrained in [False, True]:
        for pretrained in [False]:
            
            prod_model, state = setup_training(config, setup_rng, init_x, init_y)
            print(rng)
            print(state.params)
            config.alpha = alpha
            config.pretrained = pretrained

            if pretrained:
                state = state.replace(params=ens_state.params)

            _, prod_state = train_loop(
                    prod_model, state, config, rng, 
                partial(make_prod_loss, per_member_loss=alpha), 
                partial(make_prod_loss, per_member_loss=alpha), 
                train_loader, val_loader,
                # test_loader,
                wandb_kwargs={
                    'mode': 'offline',
                    # 'notes': 'pre-trained',
                    # 'tags': ['MNIST testing', 'pre-trained'],
                },
                # plot_fn=make_plots,
                # plot_freq=1,
            )
            
            pretrained = "_pretrained" if pretrained else ""
            save_checkpoint(f'dynNN_redux/prod_model_{i}_{alpha}{pretrained}', prod_state, 1, overwrite=True)

+---------------------------------------------+------------+---------+-----------+--------+
| Name                                        | Shape      | Size    | Mean      | Std    |
+---------------------------------------------+------------+---------+-----------+--------+
| batch_stats/nets_0/layer_0/BatchNorm_0/mean | (200,)     | 200     | 0.0       | 0.0    |
| batch_stats/nets_0/layer_0/BatchNorm_0/var  | (200,)     | 200     | 1.0       | 0.0    |
| batch_stats/nets_0/layer_1/BatchNorm_0/mean | (200,)     | 200     | 0.0       | 0.0    |
| batch_stats/nets_0/layer_1/BatchNorm_0/var  | (200,)     | 200     | 1.0       | 0.0    |
| batch_stats/nets_0/layer_2/BatchNorm_0/mean | (200,)     | 200     | 0.0       | 0.0    |
| batch_stats/nets_0/layer_2/BatchNorm_0/var  | (200,)     | 200     | 1.0       | 0.0    |
| batch_stats/nets_0/layer_3/BatchNorm_0/mean | (200,)     | 200     | 0.0       | 0.0    |
| batch_stats/nets_0/layer_3/BatchNorm_0/var  | (200,)     | 200     | 1.0      

TrainState(step=0, apply_fn=<bound method Module.apply of Cls_Ens(
    # attributes
    size = 5
    net = {'depth': 5, 'hidden_size': 200, 'out_size': 10, 'p_drop': 0.1}
    weights_init = ones
    logscale_init = zeros
    learn_weights = False
)>, params=FrozenDict({
    weights: DeviceArray([1., 1., 1., 1., 1.], dtype=float32),
    nets_0: {
        input_layer: {
            kernel: DeviceArray([[-0.03201382, -0.02453079,  0.03291668, ...,  0.02581717,
                          -0.00248282,  0.01266668],
                         [ 0.00100428, -0.0133006 , -0.03137461, ..., -0.03330732,
                          -0.00251798,  0.03095361],
                         [-0.02253419,  0.00236716, -0.01528232, ..., -0.00299587,
                           0.00240065,  0.02457258],
                         ...,
                         [-0.02472373, -0.00661086,  0.02736293, ...,  0.03323269,
                           0.02774817,  0.01217405],
                         [-0.00975033,  0.02644

+---------------------------------------------+------------+---------+-----------+--------+
| Name                                        | Shape      | Size    | Mean      | Std    |
+---------------------------------------------+------------+---------+-----------+--------+
| batch_stats/nets_0/layer_0/BatchNorm_0/mean | (200,)     | 200     | 0.0       | 0.0    |
| batch_stats/nets_0/layer_0/BatchNorm_0/var  | (200,)     | 200     | 1.0       | 0.0    |
| batch_stats/nets_0/layer_1/BatchNorm_0/mean | (200,)     | 200     | 0.0       | 0.0    |
| batch_stats/nets_0/layer_1/BatchNorm_0/var  | (200,)     | 200     | 1.0       | 0.0    |
| batch_stats/nets_0/layer_2/BatchNorm_0/mean | (200,)     | 200     | 0.0       | 0.0    |
| batch_stats/nets_0/layer_2/BatchNorm_0/var  | (200,)     | 200     | 1.0       | 0.0    |
| batch_stats/nets_0/layer_3/BatchNorm_0/mean | (200,)     | 200     | 0.0       | 0.0    |
| batch_stats/nets_0/layer_3/BatchNorm_0/var  | (200,)     | 200     | 1.0      

TrainState(step=0, apply_fn=<bound method Module.apply of Hard_OvR_Ens(
    # attributes
    size = 5
    net = {'depth': 5, 'hidden_size': 200, 'out_size': 10, 'p_drop': 0.1}
    weights_init = ones
    logscale_init = zeros
    learn_weights = False
)>, params=FrozenDict({
    weights: DeviceArray([1., 1., 1., 1., 1.], dtype=float32),
    nets_0: {
        input_layer: {
            kernel: DeviceArray([[-0.03201382, -0.02453079,  0.03291668, ...,  0.02581717,
                          -0.00248282,  0.01266668],
                         [ 0.00100428, -0.0133006 , -0.03137461, ..., -0.03330732,
                          -0.00251798,  0.03095361],
                         [-0.02253419,  0.00236716, -0.01528232, ..., -0.00299587,
                           0.00240065,  0.02457258],
                         ...,
                         [-0.02472373, -0.00661086,  0.02736293, ...,  0.03323269,
                           0.02774817,  0.01217405],
                         [-0.00975033,  0.

2022-10-14 16:39:46.104081: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-14 16:39:46.677137: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-14 16:39:46.677203: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-10-14 16:39:46.677211: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


  0%|          | 0/50 [00:00<?, ?it/s]


KeyboardInterrupt



### 3) metrics

In [ ]:
results_df = pd.DataFrame(columns=['model_name', 'n_members', 'random_seed', 'H', 'err', 'brier', 'nll'])
results_df

In [ ]:
s = set(range(config.model.size))
power_set = sum(map(lambda r: list(combinations(s, r)), range(1, len(s)+1)), [])

In [ ]:
X_test, y_test = list(zip(*test_loader.dataset))

In [ ]:
def compute_metrics(model, model_name, state, random_seed):
    pred_fun = partial(
            model.apply,
            {"params": state['params'], **state['model_state']},
            train=False,
            method=model.ens_logits
        )
    logits = jax.vmap(
        pred_fun, axis_name="batch"
    )(jnp.array(X_test))

    rows = []
    for indices in power_set:
        n_members = len(indices)
        logits_ = logits[:, indices, :]

        if 'prod' in model_name:
            entropies = jax.vmap(ovr_entropy)(logits_)
            nlls_ = jax.vmap(ovr_nll)(logits_, jnp.array(y_test))
            infs = jnp.isinf(nlls_)
            print(f"dropping {infs.sum()} infs for prod of {n_members}")
            nlls = nlls_[~infs]
            briers = jax.vmap(ovr_brier)(logits_, jnp.array(y_test))
            errs = jax.vmap(ovr_err)(logits_, jnp.array(y_test))
        elif 'ens' in model_name:
            entropies = jax.vmap(categorical_entropy)(logits_)
            nlls_ = jax.vmap(categorical_nll)(logits_, jnp.array(y_test))
            infs = jnp.isinf(nlls_)
            if infs.sum() > 0:
                print(f"dropping {infs.sum()} infs for Ens of {n_members}")
                print(logits_[infs])
                print(jnp.array(y_test)[infs])
            nlls = nlls_[~infs]
            briers = jax.vmap(categorical_brier)(logits_, jnp.array(y_test))
            errs = jax.vmap(categorical_err)(logits_, jnp.array(y_test))
        else:
            raise ValueError()
            
        results = {'model_name': model_name,
                    'n_members': n_members,
                    'random_seed': random_seed,
                    'H': entropies.mean(),
                    'nll': nlls.mean(),
                    'err': errs.mean(),
                    'brier': briers.mean(),
                }

        rows.append(results)
    return rows

In [ ]:
models = [('ens', None, None)] + [('prod', alpha, pretrained) for alpha in ALPHAS for pretrained in [True, False]]

In [ ]:
models

In [ ]:
def generate_name(model_type, alpha, pretrained, i):
    if model_type == "ens":
        return f"{model_type}_model_{i}"
    else:
        pretrained = "_pretrained" if pretrained else ""
        return f"{model_type}_model_{i}_{alpha}{pretrained}"

In [ ]:
for model_type, alpha, pretrained in models:
    for i in range(1):
        model_name = generate_name(model_type, alpha, pretrained, i)
        print(model_name)
        model = prod_model if model_type == 'prod' else ens_model
        state = restore_checkpoint(f'dynNN_redux/{model_name}', 1)
        
        results = compute_metrics(model, model_name, state, i)
        results_df = pd.concat([
            results_df,
            pd.DataFrame(results)],
            ignore_index=True
        )

In [ ]:
pd.set_option('display.max_rows', 500)
results_df

In [ ]:
min_mse_df = results_df[results_df.n_members == config.model.size][['model_name', 'random_seed', 'err', 'nll', 'brier']].rename(
    columns={'err': 'final_err', 'nll': 'final_nll', 'brier': 'final_brier'}
)
min_mse_df

In [ ]:
tmp_df = results_df.merge(min_mse_df, on=['model_name', 'random_seed'], how='left')
tmp_df['err_diff'] = tmp_df['err'] - tmp_df['final_err'] 
tmp_df['nll_diff'] = tmp_df['nll'] - tmp_df['final_nll'] 
tmp_df['brier_diff'] = tmp_df['brier'] - tmp_df['final_brier'] 
tmp_df

In [ ]:
agg_df = tmp_df.groupby(by=['model_name', 'n_members']).agg({
    'H': ['mean', 'std', 'count'],
    'err_diff': ['mean', 'std', 'count'],
    'err': ['mean', 'std', 'count'],
    'nll_diff': ['mean', 'std', 'count'],
    'nll': ['mean', 'std', 'count'],
    'brier_diff': ['mean', 'std', 'count'],
    'brier': ['mean', 'std', 'count'],
})
agg_df

In [ ]:
agg_df["err"]["mean"]

In [ ]:
agg_df["err"]["mean"].loc[:, 1]

In [ ]:
agg_df["err"]["mean"].loc[:, 5]

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5), sharey=True)

agg_df["err"]["mean"].loc['ens_model_0', :].plot(ax=axes[0], label="DE")
agg_df["err"]["mean"].loc['ens_model_0', :].plot(ax=axes[1], label="DE")

alpha_plot = np.linspace(0.2, 0.9, len(ALPHAS))
for i, alpha in enumerate(ALPHAS):
    agg_df["err"]["mean"].loc[f'prod_model_0_{alpha}', :].plot(ax=axes[0], 
                                                               label=alpha, 
                                                               color="red", 
                                                               alpha=alpha_plot[i])
    agg_df["err"]["mean"].loc[f'prod_model_0_{alpha}_pretrained', :].plot(ax=axes[1], 
                                                                          label=alpha,
                                                                          color="red", 
                                                                          alpha=alpha_plot[i])
    
axes[0].set_title("w/o pretraining")
axes[1].set_title("with pretraining")
    
plt.legend()
plt.show()

In [ ]:
fig = make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=("with pretraining", "w/o pretraining"))

fig.add_trace(
    go.Scatter(x=list(agg_df["err"]["mean"].loc['ens_model_0', :].index), 
               y=list(agg_df["err"]["mean"].loc['ens_model_0', :].values), 
               name="DE", marker = {'color' : 'blue'}),
    row=1, col=1,
)

fig.add_trace(
    go.Scatter(x=list(agg_df["err"]["mean"].loc['ens_model_0', :].index),
               y=list(agg_df["err"]["mean"].loc['ens_model_0', :].values),
               showlegend=False, name="DE", marker = {'color' : 'blue'}),
    row=1, col=2, 
)

alpha_plot = np.linspace(0.2, 0.9, len(ALPHAS))
for i, alpha in enumerate(ALPHAS):
    fig.add_trace(
    go.Scatter(x=list(agg_df["err"]["mean"].loc[f'prod_model_0_{alpha}', :].index), 
               y=list(agg_df["err"]["mean"].loc[f'prod_model_0_{alpha}', :].values), 
               name=alpha, marker = {'color' : 'red'}, opacity=alpha_plot[i]),
    row=1, col=1,
)

    fig.add_trace(
        go.Scatter(x=list(agg_df["err"]["mean"].loc[f'prod_model_0_{alpha}_pretrained', :].index),
                   y=list(agg_df["err"]["mean"].loc[f'prod_model_0_{alpha}_pretrained', :].values), 
                   name=alpha, marker = {'color' : 'red'}, showlegend=False, opacity=alpha_plot[i]),
        row=1, col=2, 
    )
    
fig.update_layout(height=600, width=1200)
fig.show()